In [ ]:
#python のバージョンの表示
import sys
print(sys.verion)

AttributeError: module 'sys' has no attribute 'vertion'

In [1]:
from ai import ai_match, ai2s

ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:15<00:00, 323.25it/s]

count     win    lose    draw
o        2969    1426     605
x        1477    2897     626
total    4446    4323    1231

ratio     win    lose    draw
o       59.4%   28.5%   12.1%
x       29.5%   57.9%   12.5%
total   44.5%   43.2%   12.3%



[('count',
  [{'win': 2969, 'lose': 1426, 'draw': 605},
   {'win': 1477, 'lose': 2897, 'draw': 626},
   {'win': 4446, 'lose': 4323, 'draw': 1231}],
  '7d'),
 ('ratio',
  [{'win': 0.5938, 'lose': 0.2852, 'draw': 0.121},
   {'win': 0.2954, 'lose': 0.5794, 'draw': 0.1252},
   {'win': 0.4446, 'lose': 0.4323, 'draw': 0.1231}],
  '7.1%')]

In [2]:
from marubatsu import Marubatsu

def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 〇 の勝利の判定
    if self.is_winner(Marubatsu.CIRCLE):
        return Marubatsu.CIRCLE
    # × の勝利の判定
    elif self.is_winner(Marubatsu.CROSS):
        return Marubatsu.CROSS   
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [3]:
from test import test_judge

test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [4]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:05<00:00, 876.84it/s] 

count     win    lose    draw
o        2942    1423     635
x        1436    2907     657
total    4378    4330    1292

ratio     win    lose    draw
o       58.8%   28.5%   12.7%
x       28.7%   58.1%   13.1%
total   43.8%   43.3%   12.9%



[('count',
  [{'win': 2942, 'lose': 1423, 'draw': 635},
   {'win': 1436, 'lose': 2907, 'draw': 657},
   {'win': 4378, 'lose': 4330, 'draw': 1292}],
  '7d'),
 ('ratio',
  [{'win': 0.5884, 'lose': 0.2846, 'draw': 0.127},
   {'win': 0.2872, 'lose': 0.5814, 'draw': 0.1314},
   {'win': 0.4378, 'lose': 0.433, 'draw': 0.1292}],
  '7.1%')]

In [5]:
def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [6]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [7]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:04<00:00, 1242.03it/s]

count     win    lose    draw
o        2976    1395     629
x        1419    2961     620
total    4395    4356    1249

ratio     win    lose    draw
o       59.5%   27.9%   12.6%
x       28.4%   59.2%   12.4%
total   44.0%   43.6%   12.5%



[('count',
  [{'win': 2976, 'lose': 1395, 'draw': 629},
   {'win': 1419, 'lose': 2961, 'draw': 620},
   {'win': 4395, 'lose': 4356, 'draw': 1249}],
  '7d'),
 ('ratio',
  [{'win': 0.5952, 'lose': 0.279, 'draw': 0.1258},
   {'win': 0.2838, 'lose': 0.5922, 'draw': 0.124},
   {'win': 0.4395, 'lose': 0.4356, 'draw': 0.1249}],
  '7.1%')]

In [8]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [9]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [10]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [11]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:02<00:00, 1891.24it/s]

count     win    lose    draw
o        2969    1468     563
x        1468    2911     621
total    4437    4379    1184

ratio     win    lose    draw
o       59.4%   29.4%   11.3%
x       29.4%   58.2%   12.4%
total   44.4%   43.8%   11.8%



[('count',
  [{'win': 2969, 'lose': 1468, 'draw': 563},
   {'win': 1468, 'lose': 2911, 'draw': 621},
   {'win': 4437, 'lose': 4379, 'draw': 1184}],
  '7d'),
 ('ratio',
  [{'win': 0.5938, 'lose': 0.2936, 'draw': 0.1126},
   {'win': 0.2936, 'lose': 0.5822, 'draw': 0.1242},
   {'win': 0.4437, 'lose': 0.4379, 'draw': 0.1184}],
  '7.1%')]

In [12]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    self.rowcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.colcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.diacount = {
        Marubatsu.CIRCLE: [0] * 2,
        Marubatsu.CROSS: [0] * 2,
    }
    
Marubatsu.restart = restart

In [13]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.colcount[self.last_turn][x] += 1
        self.rowcount[self.last_turn][y] += 1
        if x == y:
            self.diacount[self.last_turn][0] += 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] += 1        
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [14]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        self.colcount[self.last_turn][x] -= 1
        self.rowcount[self.last_turn][y] -= 1
        if x == y:
            self.diacount[self.last_turn][0] -= 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] -= 1           
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        
        self.last_move = self.records[-1]  
        
Marubatsu.unmove = unmove

In [15]:
def is_winner(self, player):
    x, y = self.last_move
    if self.rowcount[player][y] == self.BOARD_SIZE or \
       self.colcount[player][x] == self.BOARD_SIZE:
        return True
    # 左上から右下方向の判定
    if x == y and self.diacount[player][0] == self.BOARD_SIZE:
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.diacount[player][1] == self.BOARD_SIZE:
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [16]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [17]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:02<00:00, 1992.20it/s]

count     win    lose    draw
o        2944    1419     637
x        1457    2917     626
total    4401    4336    1263

ratio     win    lose    draw
o       58.9%   28.4%   12.7%
x       29.1%   58.3%   12.5%
total   44.0%   43.4%   12.6%



[('count',
  [{'win': 2944, 'lose': 1419, 'draw': 637},
   {'win': 1457, 'lose': 2917, 'draw': 626},
   {'win': 4401, 'lose': 4336, 'draw': 1263}],
  '7d'),
 ('ratio',
  [{'win': 0.5888, 'lose': 0.2838, 'draw': 0.1274},
   {'win': 0.2914, 'lose': 0.5834, 'draw': 0.1252},
   {'win': 0.4401, 'lose': 0.4336, 'draw': 0.1263}],
  '7.1%')]

In [18]:
def judge(self):
    if self.move_count == 0:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [19]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [20]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:03<00:00, 1661.86it/s]

count     win    lose    draw
o        2916    1439     645
x        1489    2903     608
total    4405    4342    1253

ratio     win    lose    draw
o       58.3%   28.8%   12.9%
x       29.8%   58.1%   12.2%
total   44.0%   43.4%   12.5%



[('count',
  [{'win': 2916, 'lose': 1439, 'draw': 645},
   {'win': 1489, 'lose': 2903, 'draw': 608},
   {'win': 4405, 'lose': 4342, 'draw': 1253}],
  '7d'),
 ('ratio',
  [{'win': 0.5832, 'lose': 0.2878, 'draw': 0.129},
   {'win': 0.2978, 'lose': 0.5806, 'draw': 0.1216},
   {'win': 0.4405, 'lose': 0.4342, 'draw': 0.1253}],
  '7.1%')]

In [21]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [22]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [23]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [24]:
def unmove(self):
    if self.move_count > 0:
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        self.last_move = self.records[-1]  

Marubatsu.unmove = unmove

In [25]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [26]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:04<00:00, 1165.33it/s]

count     win    lose    draw
o        2964    1391     645
x        1422    2938     640
total    4386    4329    1285

ratio     win    lose    draw
o       59.3%   27.8%   12.9%
x       28.4%   58.8%   12.8%
total   43.9%   43.3%   12.8%



[('count',
  [{'win': 2964, 'lose': 1391, 'draw': 645},
   {'win': 1422, 'lose': 2938, 'draw': 640},
   {'win': 4386, 'lose': 4329, 'draw': 1285}],
  '7d'),
 ('ratio',
  [{'win': 0.5928, 'lose': 0.2782, 'draw': 0.129},
   {'win': 0.2844, 'lose': 0.5876, 'draw': 0.128},
   {'win': 0.4386, 'lose': 0.4329, 'draw': 0.1285}],
  '7.1%')]